In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as Data
import torchvision
from torchvision import transforms, datasets
import matplotlib.pyplot as plt
import numpy as np
import pickle

In [2]:
torch.manual_seed(42)

transform = transforms.Compose([
    transforms.RandomRotation(degrees=30),
    transforms.RandomResizedCrop(128, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

In [3]:
train_dataset = datasets.ImageFolder(root=r'C:\Guvi_final_project\Image Classification\dataset\training_set', transform=transform)
train_loader = Data.DataLoader(train_dataset, batch_size=4, shuffle=True)

test_dataset = datasets.ImageFolder(root=r'C:\Guvi_final_project\Image Classification\dataset\test_set', transform=transform)
test_loader = Data.DataLoader(test_dataset, batch_size=4, shuffle=False)

In [4]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(64 * 16 * 16, 512)
        self.fc2 = nn.Linear(512, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 64 * 16 * 16)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [5]:
model = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),momentum=0.9,lr=0.001)
exp_lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)

In [6]:
import time

st=time.time()

train_acc0=[]
test_acc0=[]

num_epochs = 30

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        if (i + 1) % 500 == 0:
            print(f'Epoch {epoch + 1}/{num_epochs}, Step [{i + 1}/{len(train_loader)}], Loss: {loss.item():.4f}')

    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    test_acc0.append(100 * correct / total)
    print(f'Accuracy on the test images: {100 * correct / total}%')

    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in train_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    train_acc0.append(100 * correct / total)
    print(f'Accuracy on the train images: {100 * correct / total}%')

    with open('Model0_test.pkl','wb') as file:
            pickle.dump(test_acc0,file)
    with open('Model0_train.pkl','wb') as file:
            pickle.dump(train_acc0,file)
    with open('Model0_test.pkl','rb') as file:
            test_acc0=pickle.load(file)
    with open('Model0_train.pkl','rb') as file:
            train_acc0=pickle.load(file)

ed=time.time()
print('Training finished in '+str(ed-st)+' Seconds!')


Epoch 1/30, Step [500/2000], Loss: 0.6885
Epoch 1/30, Step [1000/2000], Loss: 0.6577
Epoch 1/30, Step [1500/2000], Loss: 0.6013
Epoch 1/30, Step [2000/2000], Loss: 0.5936
Accuracy on the test images: 63.65%
Accuracy on the train images: 63.2375%
Epoch 2/30, Step [500/2000], Loss: 0.5968
Epoch 2/30, Step [1000/2000], Loss: 0.4853
Epoch 2/30, Step [1500/2000], Loss: 0.7412
Epoch 2/30, Step [2000/2000], Loss: 0.4338
Accuracy on the test images: 66.2%
Accuracy on the train images: 67.775%
Epoch 3/30, Step [500/2000], Loss: 0.9026
Epoch 3/30, Step [1000/2000], Loss: 1.3741
Epoch 3/30, Step [1500/2000], Loss: 1.0785
Epoch 3/30, Step [2000/2000], Loss: 0.3481
Accuracy on the test images: 71.2%
Accuracy on the train images: 71.2%
Epoch 4/30, Step [500/2000], Loss: 0.7923
Epoch 4/30, Step [1000/2000], Loss: 0.5486
Epoch 4/30, Step [1500/2000], Loss: 0.6316
Epoch 4/30, Step [2000/2000], Loss: 0.4230
Accuracy on the test images: 69.1%
Accuracy on the train images: 69.975%
Epoch 5/30, Step [500/20

In [7]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(64 * 16 * 16, 512)
        self.dropout = nn.Dropout(p=0.2)
        self.fc2 = nn.Linear(512, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 64 * 16 * 16)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [8]:
model = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),momentum=0.9,lr=0.001)
exp_lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)

In [9]:
import time

st=time.time()

train_acc1=[]
test_acc1=[]

num_epochs = 30

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        if (i + 1) % 500 == 0:
            print(f'Epoch {epoch + 1}/{num_epochs}, Step [{i + 1}/{len(train_loader)}], Loss: {loss.item():.4f}')

    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    test_acc1.append(100 * correct / total)
    print(f'Accuracy on the test images: {100 * correct / total}%')

    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in train_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    train_acc1.append(100 * correct / total)
    print(f'Accuracy on the train images: {100 * correct / total}%')

    with open('Model1_test.pkl','wb') as file:
            pickle.dump(test_acc1,file)
    with open('Model1_train.pkl','wb') as file:
            pickle.dump(train_acc1,file)
    with open('Model1_test.pkl','rb') as file:
            test_acc1=pickle.load(file)
    with open('Model1_train.pkl','rb') as file:
            train_acc1=pickle.load(file)

ed=time.time()
print('Training finished in '+str(ed-st)+' Seconds!')

Epoch 1/30, Step [500/2000], Loss: 0.7244
Epoch 1/30, Step [1000/2000], Loss: 0.6502
Epoch 1/30, Step [1500/2000], Loss: 0.5827
Epoch 1/30, Step [2000/2000], Loss: 0.6553
Accuracy on the test images: 64.2%
Accuracy on the train images: 63.45%
Epoch 2/30, Step [500/2000], Loss: 0.7184
Epoch 2/30, Step [1000/2000], Loss: 0.8004
Epoch 2/30, Step [1500/2000], Loss: 0.6926
Epoch 2/30, Step [2000/2000], Loss: 0.7954
Accuracy on the test images: 65.4%
Accuracy on the train images: 66.475%
Epoch 3/30, Step [500/2000], Loss: 0.6326
Epoch 3/30, Step [1000/2000], Loss: 0.4555
Epoch 3/30, Step [1500/2000], Loss: 0.6998
Epoch 3/30, Step [2000/2000], Loss: 0.5938
Accuracy on the test images: 69.2%
Accuracy on the train images: 71.3625%
Epoch 4/30, Step [500/2000], Loss: 0.5684
Epoch 4/30, Step [1000/2000], Loss: 1.0297
Epoch 4/30, Step [1500/2000], Loss: 0.5295
Epoch 4/30, Step [2000/2000], Loss: 0.5237
Accuracy on the test images: 73.0%
Accuracy on the train images: 72.5125%
Epoch 5/30, Step [500/2

In [10]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(64 * 16 * 16, 512)
        self.dropout = nn.Dropout(p=0.4)
        self.fc2 = nn.Linear(512, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 64 * 16 * 16)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [11]:
model = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),momentum=0.9,lr=0.001)
exp_lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)

In [12]:
import time

st=time.time()

train_acc2=[]
test_acc2=[]

num_epochs = 30

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        if (i + 1) % 500 == 0:
            print(f'Epoch {epoch + 1}/{num_epochs}, Step [{i + 1}/{len(train_loader)}], Loss: {loss.item():.4f}')

    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    test_acc2.append(100 * correct / total)
    print(f'Accuracy on the test images: {100 * correct / total}%')

    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in train_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    train_acc2.append(100 * correct / total)
    print(f'Accuracy on the train images: {100 * correct / total}%')
    
    with open('Model2_test.pkl','wb') as file:
            pickle.dump(test_acc2,file)
    with open('Model2_train.pkl','wb') as file:
            pickle.dump(train_acc2,file)
    with open('Model2_test.pkl','rb') as file:
            test_acc2=pickle.load(file)
    with open('Model2_train.pkl','rb') as file:
            train_acc2=pickle.load(file)

ed=time.time()
print('Training finished in '+str(ed-st)+' Seconds!')

Epoch 1/30, Step [500/2000], Loss: 0.7229
Epoch 1/30, Step [1000/2000], Loss: 0.6508
Epoch 1/30, Step [1500/2000], Loss: 0.7061
Epoch 1/30, Step [2000/2000], Loss: 0.6967
Accuracy on the test images: 62.25%
Accuracy on the train images: 61.4375%
Epoch 2/30, Step [500/2000], Loss: 0.4684
Epoch 2/30, Step [1000/2000], Loss: 0.8348
Epoch 2/30, Step [1500/2000], Loss: 0.7173
Epoch 2/30, Step [2000/2000], Loss: 0.5185
Accuracy on the test images: 67.8%
Accuracy on the train images: 68.6125%
Epoch 3/30, Step [500/2000], Loss: 0.6293
Epoch 3/30, Step [1000/2000], Loss: 0.3898
Epoch 3/30, Step [1500/2000], Loss: 0.8413
Epoch 3/30, Step [2000/2000], Loss: 0.5024
Accuracy on the test images: 71.2%
Accuracy on the train images: 71.9875%
Epoch 4/30, Step [500/2000], Loss: 0.4125
Epoch 4/30, Step [1000/2000], Loss: 0.2607
Epoch 4/30, Step [1500/2000], Loss: 0.5640
Epoch 4/30, Step [2000/2000], Loss: 0.5156
Accuracy on the test images: 70.25%
Accuracy on the train images: 72.2375%
Epoch 5/30, Step [